In [8]:
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd

# 1. Data - 5 ans d'AAPL
data = yf.download("AAPL", start="2019-01-01", progress=False)[
            ["Open", "High", "Low", "Close", "Volume"]
        ].dropna().droplevel(1, axis=1)  # remove multi-index
data.index.name = "Date"

# 2. Petit indicateur : SMA20
# def SMA(s, n=20): return s.rolling(n).mean()
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class MultiTradeHold5(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    sma_len   = 20
    # -----------------------------------

    def init(self):
        self.sma = self.I(SMA, self.data.Close, self.sma_len)

    def next(self):
        # ❶ Fermer tous les trades qui ont 5 barres d'âge
        for trade in list(self.trades):   
                            # toutes positions ouvertes
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # ❷ Signal d'entrée : cross Close ↗ SMA20  (long uniquement pour l'exemple)
        if crossover(self.data.Close, self.sma):
            # aucune vérification self.position → on empile librement
            self.buy(size=1)

# 3. Lancement du back-test
bt = Backtest(data, MultiTradeHold5,
              cash=10_000, commission=0.002, trade_on_close=True)
stats = bt.run()

# Visualize the backtest results with a plot
# Set a custom size and save to HTML file for future reference
bt.plot(plot_width=1200, filename="apple_backtest_results.html")


/Users/davidzhu/Local/gestion-quantitative/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:701: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/davidzhu/Local/gestion-quantitative/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:701: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2753', ...)